---
comments: false
layout: post
title: testing
description: Testing2
type: hacks
courses: { compsci: {week: 0} }
image: /images/background1.png
sprite: /images/Robot_roders.png
---

In [19]:
%%html
<!-- Liquid code, run by Jekyll, used to define location of asset(s) -->
{% assign backgroundFile = site.baseurl | append: page.image %}
{% assign spriteImage = site.baseurl | append: page.sprite %}

<style>
    #controls {
        position: relative;
        z-index: 2; /* Ensure the controls are on top of the rob canvas */
    }

    /* Style the rob canvas to be the same size as the viewport */
    #robCanvas {
        position: absolute;
        top: 0;
        left: 0;
        width: 100%;
        height: 100%;
        z-index: 0; /* Place it below the background */
    }
</style>

<!-- Prepare DOM elements -->
<!-- Wrap both the rob canvas and controls in a container div -->
<div id="canvasContainer">
    <div id="controls"> <!-- Controls -->
        <button id="toggleCanvasEffect">Invert</button>

        <input type="radio" name="animation" id="idle">
        <label for="idle">Idle</label>
        <input type="radio" name="animation" id="barking">
        <label for="barking">Barking</label>
        <input type="radio" name="animation" id="walking" checked>
        <label for="walking">Walking</label>
    </div>
    <canvas id="backgroundID">
        <img id="backgroundImage" src="{{backgroundFile}}">
    </canvas>
</div>

<script>
/* Background part of Game
 * scrolling 
*/
// Prepare Background Image
const backgroundImg = new Image();
backgroundImg.src = '{{backgroundFile}}';  // Jekyll/Liquid puts filename here

// Prepare Sprite Image
const robImg = new Image();
robImg.src = '{{spriteImage}}';

// Prepare Canvas
const canvas = document.getElementById("backgroundID");
const ctx = canvas.getContext('2d');

// rob animation part
const robCanvas = document.createElement("canvas");
const robCtx = robCanvas.getContext("2d");

// Prepare Window extents related to viewport
const maxWidth = window.innerWidth;
const maxHeight = window.innerHeight;

backgroundImg.onload = function () {
    // Setup background constants from background image
    const WIDTH = backgroundImg.width;  // Image() width (meta data)
    const HEIGHT = backgroundImg.height; // Image() height
    const ASPECT_RATIO = WIDTH / HEIGHT;
    const ADJUST = 1.42 // visual layer adjust, use "1"" for a perfect loop 

    // Set Dimensions to match the image width
    const canvasWidth = maxWidth;
    const canvasHeight = canvasWidth / ASPECT_RATIO;  // height is oriented by width
    const canvasLeft = 0; // Start image from the left edge horizontally
    const canvasTop = (maxHeight - canvasHeight) / 2;  // center image vertically

    // Set Style properties for the background canvas
    canvas.width = WIDTH / ADJUST;
    canvas.height = HEIGHT / ADJUST;
    canvas.style.width = `${canvasWidth}px`;
    canvas.style.height = `${canvasHeight}px`;
    canvas.style.position = 'absolute';
    canvas.style.left = `${canvasLeft}px`;
    canvas.style.top = `${canvasTop}px`;

    // Game speed is a common game variable
    var gameSpeed = 2;

    // Layer is set up to support Parallax, multiple layers
    class Layer {
        constructor(image, speedRatio) {
            this.x = 0;
            this.y = 0;
            this.width = WIDTH;
            this.height = HEIGHT;
            this.image = image;
            this.speedRatio = speedRatio;
            this.speed = gameSpeed * this.speedRatio;
            this.frame = 0;
        }
        update() {
            this.x = (this.x - this.speed) % this.width;
        }
        draw() {
            ctx.drawImage(this.image, this.x, this.y);
            ctx.drawImage(this.image, this.x + this.width, this.y);
        }
    }

    // Setup rob sprite constraints
    const SPRITE_WIDTH = 61;  // matches sprite pixel width
    const SPRITE_HEIGHT = 65; // matches sprite pixel height
    const SPRITE_FRAMES = 5;  // matches number of frames per sprite row; this code assumes each row is the same
    const SPRITE_SCALE = 1;  // controls the size of the sprite on the canvas

    class rob extends Layer {
        constructor(image, speedRatio) {
            super(image, speedRatio);
            this.minFrame = 0;
            this.maxFrame = SPRITE_FRAMES;
            this.frameX = 0;
            this.frameY = 2;  // walking as default
            this.robX = canvasWidth; // Initialize the rob's x position to the right edge of the canvas
        }
    
        update() {
            if (this.frameY == 2) {
                this.robX -= this.speed;  // Move the rob to the left
                // Check if the rob has moved off the left edge of the canvas
                if (this.robX < -robCanvas.width) {
                    this.robX = canvasWidth; // Reset the rob's x position to the right edge
                }
            }
            // Update frameX of the object
            if (this.frameX < this.maxFrame) {
                this.frameX++;
            } else {
                this.frameX = 0;
            }
        }
    
        // Draw rob object
        draw() {
            // Set fixed dimensions and position for the robCanvas
            robCanvas.width = SPRITE_WIDTH * SPRITE_SCALE;
            robCanvas.height = SPRITE_HEIGHT * SPRITE_SCALE;
            robCanvas.style.width = `${robCanvas.width}px`;
            robCanvas.style.height = `${robCanvas.height}px`;
            robCanvas.style.position = 'absolute';
            robCanvas.style.left = `${this.robX}px`; // Set the rob's left position based on its x-coordinate
            robCanvas.style.top = `${canvasHeight}px`;
    
            robCtx.drawImage(
                this.image,
                this.frameX * SPRITE_WIDTH,
                this.frameY * SPRITE_HEIGHT,
                SPRITE_WIDTH,
                SPRITE_HEIGHT,
                0,
                0,
                robCanvas.width,
                robCanvas.height
            );
        }
    }
    

    // Background object
    var backgroundObj = new Layer(backgroundImg, 0.2);
    var robObj = new rob(robImg, 0.5);

    // Append the rob canvas to the body
    document.body.appendChild(robCanvas);

    // Animation loop
    function animation() {
        backgroundObj.update();
        backgroundObj.draw();

        robObj.update();
        robObj.draw();

        requestAnimationFrame(animation);  // cycle animation, recursion
    }

    // Start animation process
    animation();

    /* Toggle "canvas filter property" 
    * look in _sass/minima/dark-mode.scss
    */
    var isFilterEnabled = true;
    const defaultFilter = getComputedStyle(document.documentElement).getPropertyValue('--default-canvas-filter');
    toggleCanvasEffect.addEventListener("click", function () {
        if (isFilterEnabled) {
            canvas.style.filter = "none";  // remove filter
            robCanvas.style.filter = "none";
        } else {
            canvas.style.filter = defaultFilter; // Apply the default filter value
            robCanvas.style.filter = defaultFilter; 
        }

        isFilterEnabled = !isFilterEnabled;  // switch boolean value
    });
    /* Control "rob action" 
     * changes y value, the row in sprite
    */
    // update frameY of rob object, action from idle, bark, walk radio control
    const controls = document.getElementById('controls');
    controls.addEventListener('click', function (event) {
        if (event.target.tagName === 'INPUT') {
            const selectedAnimation = event.target.id;
            switch (selectedAnimation) {
                case 'idle':
                    robObj.frameY = 0;
                    break;
                case 'barking':
                    robObj.frameY = 1;
                    break;
                case 'walking':
                    robObj.frameY = 2;
                    break;
                default:
                    break;
            }
        }
    });
};
</script>